In [91]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

In [101]:
mnist_dataset = datasets.fetch_openml('mnist_784')

random_state = 0


X = mnist_dataset.data
y = mnist_dataset.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [108]:
print(mnist_dataset.DESCR)

**Author**: Yann LeCun, Corinna Cortes, Christopher J.C. Burges  
**Source**: [MNIST Website](http://yann.lecun.com/exdb/mnist/) - Date unknown  
**Please cite**:  

The MNIST database of handwritten digits with 784 features, raw data available at: http://yann.lecun.com/exdb/mnist/. It can be split in a training set of the first 60,000 examples, and a test set of 10,000 examples  

It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image. It is a good database for people who want to try learning techniques and pattern recognition methods on real-world data while spending minimal efforts on preprocessing and formatting. The original black and white (bilevel) images from NIST were size normalized to fit in a 20x20 pixel box while preserving their aspect ratio. The resulting images contain grey levels as a result of the anti-aliasing technique used by the normalization algorithm. the images were centered in a 28x28 image b

In [102]:
# Reduce dimension to 2 with PCA
pca = make_pipeline(StandardScaler(),
                    PCA(0.95))

# Use a nearest neighbor classifier to evaluate the methods
decision_tree_classifier = DecisionTreeClassifier(max_depth=10, min_samples_split=5)

dim_reduction_methods = [('PCA', pca)]


In [103]:
for i, (name, model) in enumerate(dim_reduction_methods):
    # Fit the method's model
    model.fit(X_train, y_train)

    # Fit a decision tree classifier on the training set
    decision_tree_classifier.fit(model.transform(X_train), y_train)

    accuracy_on_tranining = accuracy_score(decision_tree_classifier.predict(model.fit_transform(X_train)), y_train)
    
    
    accuracy_on_testing = accuracy_score(decision_tree_classifier.predict(model.transform(X_test)), y_test)
    
    print('Training accuracy ', accuracy_on_tranining)
    print('Testing accuracy ',  accuracy_on_testing)

Training accuracy  0.8488367346938775
Testing accuracy  0.7988571428571428
